In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print("Данные загружены!")
print(f"Размер учебника (train): {train.shape}")
print(f"Размер экзамена (test): {test.shape}")

Данные загружены!
Размер учебника (train): (700000, 26)
Размер экзамена (test): (300000, 25)


In [2]:
print(train.columns.tolist())

['id', 'age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history', 'diagnosed_diabetes']


In [6]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  heart_rate                

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [8]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

target_col = 'diagnosed_diabetes'
X = train.drop(['id', target_col], axis=1)
y = train[target_col]


for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in test.columns:
            test[col] = test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\n🏆 ROC-AUC: {score:.5f}")


🏆 ROC-AUC: 0.72191


In [9]:
X_test = test.drop(['id'], axis=1)

test_probs = model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    'id': test['id'],
    'diagnosed_diabetes': test_probs
})

submission.to_csv('submission.csv', index=False)

print("submission.csv готов")
print(submission.head())

submission.csv готов
       id  diagnosed_diabetes
0  700000            0.519037
1  700001            0.670999
2  700002            0.782870
3  700003            0.414762
4  700004            0.886695


In [10]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target_col = 'diagnosed_diabetes'

X = train.drop(['id', target_col], axis=1)
y = train[target_col]
X_test = test.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=500,      
    learning_rate=0.05,     
    max_depth=6,           
    num_leaves=31,           
    reg_lambda=10,         
    random_state=42,
    verbose=-1
)

print("running model with new features")
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
new_score = roc_auc_score(y_val, probs)

print(f"new score {new_score:.5f}")

running model with new features
new score 0.72536


In [ ]:
tuned_probs = model.predict_proba(X_test)[:, 1]

submission_tuned = pd.DataFrame({
    'id': test['id'],
    'diagnosed_diabetes': tuned_probs
})

submission_tuned.to_csv('submission_tuned.csv', index=False)


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_eng = train.copy()
test_eng = test.copy()

train_eng['Risk_Factor'] = train_eng['age'] * train_eng['bmi']
test_eng['Risk_Factor'] = test_eng['age'] * test_eng['bmi']

train_eng['Waist_Age'] = train_eng['waist_to_hip_ratio'] * train_eng['age']
test_eng['Waist_Age'] = test_eng['waist_to_hip_ratio'] * test_eng['age']

print("new features added!")

target_col = 'diagnosed_diabetes'
X = train_eng.drop(['id', target_col], axis=1)
y = train_eng[target_col]
X_test = test_eng.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=600,      
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    reg_lambda=10,
    random_state=42,
    verbose=-1
)

model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\nValidation score: {score:.5f}")

# 7. Сохраняем файл для Kaggle
final_probs = model.predict_proba(X_test)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_feature.csv', index=False)
print("submission_feature.csv готов")

In [ ]:
X_final = train_eng.copy()
X_test_final = test_eng.copy()

X_final['Cholesterol_Ratio'] = X_final['ldl_cholesterol'] / (X_final['hdl_cholesterol'] + 0.1)
X_test_final['Cholesterol_Ratio'] = X_test_final['ldl_cholesterol'] / (X_test_final['hdl_cholesterol'] + 0.1)

X_final['BP_Diff'] = X_final['systolic_bp'] - X_final['diastolic_bp']
X_test_final['BP_Diff'] = X_test_final['systolic_bp'] - X_test_final['diastolic_bp']

X_final['Total_Lipids'] = X_final['cholesterol_total'] + X_final['triglycerides']
X_test_final['Total_Lipids'] = X_test_final['cholesterol_total'] + X_test_final['triglycerides']

print("medical features added!")

target_col = 'diagnosed_diabetes'
X = X_final.drop(['id', target_col], axis=1)
y = X_final[target_col]
X_test_submit = X_test_final.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test_submit.columns:
            X_test_submit[col] = X_test_submit[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=700,
    learning_rate=0.04, 
    max_depth=7,
    num_leaves=35,
    random_state=42,
    verbose=-1
)

print("🧠 Обучение финальной модели...")
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\n: {score:.5f}")

final_probs = model.predict_proba(X_test_submit)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_medical.csv', index=False)
print("✅ Файл submission_medical.csv готов")

NameError: name 'train_eng' is not defined

In [ ]:
%pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Risk_Factor'] = train['age'] * train['bmi']
test['Risk_Factor'] = test['age'] * test['bmi']

target_col = 'diagnosed_diabetes'
X = train.drop(['id', target_col], axis=1)
y = train[target_col]
X_test = test.drop(['id'], axis=1)

cat_features = [col for col in X.columns if X[col].dtype == 'object']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model_cat = CatBoostClassifier(
    iterations=1000,        
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',       
    random_seed=42,
    verbose=100              
)

print("catboost")
model_cat.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50 
)

final_probs = model_cat.predict_proba(X_test)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_catboost.csv', index=False)
print("\n✅submission_catboost.csv готов")

In [ ]:
import pandas as pd

sub_lgbm = pd.read_csv('submission_feature.csv')
sub_cat = pd.read_csv('submission_catboost.csv')  

blend_pred = (sub_lgbm['diagnosed_diabetes'] * 0.6) + (sub_cat['diagnosed_diabetes'] * 0.4)

sub_blend = pd.DataFrame({
    'id': sub_lgbm['id'],
    'diagnosed_diabetes': blend_pred
})

sub_blend.to_csv('submission_blend.csv', index=False)
